In [1]:
import numpy as np
import pandas as pd
from statsmodels.api import OLS, add_constant

In [2]:
xls = pd.ExcelFile('./bank_datastream0816.xlsx')
for sheet_name in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet_name)
    df.to_csv(f'{sheet_name}.csv', index=False)


In [3]:
stock_data=pd.read_csv("./share price daily.csv")
market_data=pd.read_csv("./market indices_daily.csv")
riskfree_data=pd.read_csv("./EONIA.csv")

In [4]:
stock_data.head()

,Date,EBS,OBS.VI,RBI.VI,ARLN.HM,CBK.DE,PBB.BE,MBK.DE,BBVA.MC,UNIE.XC,...,ISP.MI,UCG.MI,ABN.AS,VLK.AS,AZA.ST,HOFI.ST,SEB-A.ST,SHB-A.ST,TFBANK.ST,NLB.IL
0,2019-01-02,57.700001,42.317120,15.009684,20.131157,5.346076,4.841865,7.301744,3.424062,0.962672,...,1.291749,8.020234,12.378850,10.646761,69.065323,42.700001,70.079155,74.827446,68.548180,9.270435
1,2019-01-03,57.540001,42.411366,15.284290,20.079140,5.380388,4.863699,7.260255,3.433240,0.958453,...,1.288361,7.906972,12.481354,10.517239,69.483315,42.849998,70.615601,74.888100,70.479111,9.224634
2,2019-01-04,58.790001,42.411366,15.706250,20.041981,5.696608,4.858240,7.260255,3.561362,0.968578,...,1.354629,8.378076,12.855190,10.672668,70.962372,43.099998,72.615067,76.419518,71.637665,9.296197
3,2019-01-07,61.549999,42.411366,16.000950,20.651340,5.732775,5.092965,7.343231,3.548881,0.983765,...,1.377532,8.599673,12.963724,10.776285,71.219597,45.349998,71.265839,75.600746,71.444580,9.039998
4,2019-01-08,62.700001,42.411366,16.027742,21.008039,5.870019,5.142092,7.260255,3.578984,0.976171,...,1.361609,8.458508,12.945635,10.750380,70.126381,44.450001,72.029877,75.085220,72.410034,9.031410


In [5]:
market_data

,Date,STOXX EUROPE 50,S&P EUROPE 350
0,2019-01-02,2993.18,1365.754
1,2019-01-03,2954.66,1354.539
2,2019-01-04,3041.85,1392.493
3,2019-01-07,3033.64,1387.646
4,2019-01-08,3054.94,1399.540
...,...,...,...
506,2020-12-24,3543.28,1574.632
507,2020-12-28,3575.41,1584.307
508,2020-12-29,3581.37,1596.413
509,2020-12-30,3571.59,1591.411


In [6]:
# Parameters
event_date = '2020-03-27'
estimation_window_start = -250  # Start of the estimation window
estimation_window_end = -4     # End of the estimation window
event_window_start = -3  #  days before the event
event_window_end = 3     #  days after the event
bankname = stock_data.iloc[0:0,1:]
marketindex = market_data.iloc[0:0,1:]
riskfreeindex = riskfree_data.iloc[0:0,1:]
bank_list = bankname.columns[0:].tolist()  # Convert the first row of stock_data to a list
market_list = marketindex.columns[0:].tolist() 
riskfree_list = riskfreeindex.columns[0:].tolist() 

In [7]:
print(bank_list)

['EBS', 'OBS.VI', 'RBI.VI', 'ARLN.HM', 'CBK.DE', 'PBB.BE', 'MBK.DE', 'BBVA.MC', 'UNIE.XC', 'AKTIA.HE', 'NDA-SE.ST', 'OMASP.HE', 'BNP.PA', 'CCN.PA', 'LLOY.L', 'MTRO.L', 'STB.L', 'STAN.L', 'TATT.AT', 'BIRG.IR', 'ARION.IC', 'BGN.MI', 'BMED.MI', 'BDB.MI', 'FBK.MI', 'ILTY.MI', 'ISP.MI', 'UCG.MI', 'ABN.AS', 'VLK.AS', 'AZA.ST', 'HOFI.ST', 'SEB-A.ST', 'SHB-A.ST', 'TFBANK.ST', 'NLB.IL']


In [8]:
print(market_list)

['STOXX EUROPE 50', 'S&P EUROPE 350']


In [9]:
print(riskfree_list)

['EONIARATE']


In [10]:
# Clean up column names
stock_data.columns = stock_data.columns.str.strip()
market_data.columns = market_data.columns.str.strip()
riskfree_data.columns = riskfree_data.columns.str.strip()

# Convert 'Date' column to datetime
date_column_name = 'Date'
stock_data[date_column_name] = pd.to_datetime(stock_data[date_column_name])
market_data[date_column_name] = pd.to_datetime(market_data[date_column_name])
riskfree_data[date_column_name] = pd.to_datetime(riskfree_data[date_column_name])

# Set Date as index if it's not already
stock_data.set_index(date_column_name, inplace=True)
market_data.set_index(date_column_name, inplace=True)
riskfree_data.set_index(date_column_name, inplace=True)


In [11]:
# Initialize an empty DataFrame to store daily returns
bank_daily_returns = pd.DataFrame()
market_daily_returns = pd.DataFrame()

selected_metric = 'EONIARATE'
riskfree_data_series = pd.to_numeric(riskfree_data[selected_metric], errors='coerce')
    
# Iterate over each column to calculate daily returns
for column in stock_data.columns:
    if column != date_column_name:  # Skip the Date column
        # Ensure the column data is numeric
        stock_data[column] = pd.to_numeric(stock_data[column], errors='coerce')
        
        # Calculate daily returns for this column
        bank_daily_returns[column] = stock_data[column].pct_change()
      
        # Calculate excess returns by subtracting the risk-free rate
        bank_daily_returns[column] = bank_daily_returns[column] - riskfree_data_series

# If the Date column was reset earlier, there's no need to add it back again
if date_column_name not in bank_daily_returns.columns:
    bank_daily_returns.reset_index(inplace=True)

# Reorder columns so that Date appears first
bank_daily_returns = bank_daily_returns[[date_column_name] + [col for col in bank_daily_returns.columns if col != date_column_name]]


In [12]:
print(bank_daily_returns)

          Date       EBS    OBS.VI    RBI.VI   ARLN.HM    CBK.DE    PBB.BE  \
0   2019-01-02       NaN       NaN       NaN       NaN       NaN       NaN   
1   2019-01-03  0.363227  0.368227  0.384295  0.363416  0.372418  0.370509   
2   2019-01-04  0.386724  0.365000  0.392607  0.363149  0.423773  0.363878   
3   2019-01-07  0.411947  0.365000  0.383763  0.395404  0.371349  0.413315   
4   2019-01-08  0.380684  0.362000  0.363674  0.379272  0.385940  0.371646   
..         ...       ...       ...       ...       ...       ...       ...   
506 2020-12-24  0.457166  0.470000  0.470000  0.470000  0.470000  0.470000   
507 2020-12-28  0.450097  0.468000  0.469186  0.481882  0.467627  0.448833   
508 2020-12-29  0.450082  0.472000  0.469038  0.473521  0.457075  0.470883   
509 2020-12-30  0.498400  0.472375  0.461087  0.459873  0.467348  0.461615   
510 2020-12-31  0.464559  0.498000  0.498000  0.498000  0.498000  0.498000   

       MBK.DE   BBVA.MC   UNIE.XC  ...    ISP.MI    UCG.MI    A

In [13]:
market_daily_returns = pd.DataFrame(index=market_data.index)
for column in market_data.columns:
    if column != date_column_name:  # Skip the Date column
        # Ensure the column data is numeric
        market_data[column] = pd.to_numeric(market_data[column], errors='coerce')
        
        # Calculate daily returns for this column
        market_daily_returns[column] = market_data[column].pct_change()
        
        # Calculate excess returns by subtracting the risk-free rate
        market_daily_returns[column] = market_daily_returns[column] - riskfree_data_series
        
print(market_daily_returns)

            STOXX EUROPE 50  S&P EUROPE 350
Date                                       
2019-01-02              NaN             NaN
2019-01-03         0.353131        0.357788
2019-01-04         0.394509        0.393020
2019-01-07         0.362301        0.361519
2019-01-08         0.369021        0.370571
...                     ...             ...
2020-12-24         0.471136        0.471388
2020-12-28         0.477068        0.474144
2020-12-29         0.473667        0.479641
2020-12-30         0.467269        0.466867
2020-12-31         0.492694        0.494333

[511 rows x 2 columns]


In [14]:
# Merge the two DataFrames on the Data column
merged_data=pd.merge(bank_daily_returns, market_daily_returns, on=date_column_name, suffixes=('_stock', '_market'))
print(merged_data)

          Date       EBS    OBS.VI    RBI.VI   ARLN.HM    CBK.DE    PBB.BE  \
0   2019-01-02       NaN       NaN       NaN       NaN       NaN       NaN   
1   2019-01-03  0.363227  0.368227  0.384295  0.363416  0.372418  0.370509   
2   2019-01-04  0.386724  0.365000  0.392607  0.363149  0.423773  0.363878   
3   2019-01-07  0.411947  0.365000  0.383763  0.395404  0.371349  0.413315   
4   2019-01-08  0.380684  0.362000  0.363674  0.379272  0.385940  0.371646   
..         ...       ...       ...       ...       ...       ...       ...   
506 2020-12-24  0.457166  0.470000  0.470000  0.470000  0.470000  0.470000   
507 2020-12-28  0.450097  0.468000  0.469186  0.481882  0.467627  0.448833   
508 2020-12-29  0.450082  0.472000  0.469038  0.473521  0.457075  0.470883   
509 2020-12-30  0.498400  0.472375  0.461087  0.459873  0.467348  0.461615   
510 2020-12-31  0.464559  0.498000  0.498000  0.498000  0.498000  0.498000   

       MBK.DE   BBVA.MC   UNIE.XC  ...    ABN.AS    VLK.AS    A

In [15]:
def find_data_in_relevent_days(origin_data, start, end):
    res_index = origin_data.index[origin_data['Date'] == event_date].tolist()[0]
    res_index_position = origin_data.index.get_loc(res_index) # true index: 3
    res_data = origin_data.iloc[res_index_position+start: res_index_position+end+1]
    return res_data


estimation_data = find_data_in_relevent_days(merged_data, start=estimation_window_start, end=estimation_window_end)
event_data = find_data_in_relevent_days(merged_data, start=event_window_start, end=event_window_end)

print(estimation_data)
print(event_data)



          Date       EBS    OBS.VI    RBI.VI   ARLN.HM    CBK.DE    PBB.BE  \
66  2019-04-04  0.386493  0.370123  0.376395  0.385731  0.396044  0.377786   
67  2019-04-05  0.386158  0.368000  0.371238  0.368000  0.384610  0.353903   
68  2019-04-08  0.360989  0.372119  0.359396  0.366864  0.345956  0.365532   
69  2019-04-09  0.359855  0.368000  0.382446  0.348776  0.375214  0.370693   
70  2019-04-10  0.387817  0.367000  0.364703  0.368425  0.343622  0.357152   
..         ...       ...       ...       ...       ...       ...       ...   
308 2020-03-17  0.451056  0.422958  0.450882  0.442074  0.494951  0.375630   
309 2020-03-18  0.408267  0.422774  0.354931  0.408524  0.429556  0.333774   
310 2020-03-19  0.396208  0.441000  0.473000  0.388501  0.512992  0.403687   
311 2020-03-20  0.368190  0.439000  0.556571  0.444309  0.421272  0.442101   
312 2020-03-23  0.445472  0.440000  0.426898  0.271353  0.410021  0.390541   

       MBK.DE   BBVA.MC   UNIE.XC  ...    ABN.AS    VLK.AS    A

In [16]:
# Market model: Regress stock returns on market returns
# Perform multivariate regression for all banks against the market indices
stoxx = market_list[0]
sp = market_list[1]


# Perform OLS regression for each bank against the market indices
results = {}
cumulative_abnormal_returns = pd.DataFrame()

for bank in bank_list:
    X = estimation_data[stoxx]  # Independent variables (market returns)
    X = add_constant(X)  # Add a constant (intercept)
    y = estimation_data[bank]  # Dependent variable (bank returns)
    
    
    model = OLS(y, X).fit()  # Fit the model
    
#     print(model.summary())
    
#     results = {}
    
    # Predict expected returns during the event window
    event_data[f'Expected_{bank}'] = model.predict(add_constant(event_data[toxx]))
    
    # Calculate abnormal returns
    event_data[f'Abnormal_{bank}'] = event_data[bank] - event_data[f'Expected_{bank}']
    
    # Calculate cumulative abnormal returns (CAR)
    event_data[f'CAR_{bank}'] = event_data[f'Abnormal_{bank}'].cumsum()
    
    # Save CAR
    cumulative_abnormal_returns[f'CAR_{bank}'] = event_data[f'CAR_{bank}']
    
    print(f"Abnormal Returns (AR) for {bank}:")
    print(event_data[f'Abnormal_{bank}'])
    
# Calculate the CAAR (Cumulative Average Abnormal Returns)
cumulative_abnormal_returns['CAAR'] = cumulative_abnormal_returns.mean(axis=1)




#     # Print the summary for each bank
#     print(f"Regression results for {bank}:")
#     print(model.summary())
#     print("\n" + "="*80 + "\n")
    
    # Display the results
# print("Estimation Start Date:", estimation_start_date)
# print("Estimation End Date:", estimation_end_date)


Abnormal Returns (AR) for EBS:
313    0.037467
314   -0.052266
315    0.030949
316   -0.066832
317    0.050415
318    0.053471
319   -0.024671
Name: Abnormal_EBS, dtype: float64
Abnormal Returns (AR) for OBS.VI:
313   -0.083816
314   -0.027382
315   -0.014007
316    0.045290
317   -0.010879
318   -0.005920
319    0.039330
Name: Abnormal_OBS.VI, dtype: float64
Abnormal Returns (AR) for RBI.VI:
313    0.014202
314   -0.027170
315    0.036356
316    0.002527
317   -0.056129
318   -0.034058
319    0.008438
Name: Abnormal_RBI.VI, dtype: float64
Abnormal Returns (AR) for ARLN.HM:
313    0.011087
314    0.127468
315   -0.085067
316    0.057509
317   -0.016487
318   -0.002385
319    0.027013
Name: Abnormal_ARLN.HM, dtype: float64
Abnormal Returns (AR) for CBK.DE:
313    0.011894
314    0.021037
315    0.008835
316   -0.015245
317   -0.083443
318   -0.016868
319   -0.022824
Name: Abnormal_CBK.DE, dtype: float64
Abnormal Returns (AR) for PBB.BE:
313    0.014712
314    0.041774
315   -0.027307
31

/Users/klara/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/var/folders/g0/780cxn9d3895qkc2jcrhv3gm0000gn/T/ipykernel_49361/2429191562.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  event_data[f'Expected_{bank}'] = model.predict(add_constant(event_data[stoxx]))
/var/folders/g0/780cxn9d3895qkc2jcrhv3gm0000gn/T/ipykernel_49361/2429191562.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

Abnormal Returns (AR) for BMED.MI:
313    0.031416
314    0.000239
315    0.039250
316    0.007918
317   -0.061530
318   -0.022059
319    0.035190
Name: Abnormal_BMED.MI, dtype: float64
Abnormal Returns (AR) for BDB.MI:
313   -0.110817
314   -0.027448
315   -0.013053
316    0.006991
317    0.023984
318   -0.003646
319    0.067463
Name: Abnormal_BDB.MI, dtype: float64
Abnormal Returns (AR) for FBK.MI:
313    0.004024
314   -0.047694
315    0.073657
316   -0.022177
317    0.004210
318   -0.023934
319    0.037997
Name: Abnormal_FBK.MI, dtype: float64
Abnormal Returns (AR) for ILTY.MI:
313    0.056658
314   -0.028322
315    0.002649
316    0.010592
317   -0.029797
318    0.042220
319    0.010107
Name: Abnormal_ILTY.MI, dtype: float64
Abnormal Returns (AR) for ISP.MI:
313   -0.001591
314   -0.011920
315   -0.008826
316    0.002403
317   -0.075591
318    0.003591
319    0.002284
Name: Abnormal_ISP.MI, dtype: float64
Abnormal Returns (AR) for UCG.MI:
313    0.002084
314   -0.025831
315   -0.0

/var/folders/g0/780cxn9d3895qkc2jcrhv3gm0000gn/T/ipykernel_49361/2429191562.py:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  event_data[f'Abnormal_{bank}'] = event_data[bank] - event_data[f'Expected_{bank}']
/var/folders/g0/780cxn9d3895qkc2jcrhv3gm0000gn/T/ipykernel_49361/2429191562.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  event_data[f'CAR_{bank}'] = event_data[f'Abnormal_{bank}'].cumsum()
/var/folders/g0/780cxn9d3895qkc2jcrhv3gm0000gn/T/ipykernel_49361/2429191562.py:24: PerformanceWarning: DataFrame is highly 

In [17]:
# Display CAAR for the event window
print(cumulative_abnormal_returns['CAAR'])

313   -0.024375
314   -0.015794
315   -0.013807
316   -0.002463
317   -0.031668
318   -0.031026
319   -0.022657
Name: CAAR, dtype: float64


In [18]:
print(event_data[['Date'] +[f'CAR_{bank}' for bank in bank_list]])

          Date   CAR_EBS  CAR_OBS.VI  CAR_RBI.VI  CAR_ARLN.HM  CAR_CBK.DE  \
313 2020-03-24  0.037467   -0.083816    0.014202     0.011087    0.011894   
314 2020-03-25 -0.014799   -0.111198   -0.012968     0.138555    0.032931   
315 2020-03-26  0.016150   -0.125205    0.023389     0.053488    0.041766   
316 2020-03-27 -0.050682   -0.079915    0.025915     0.110997    0.026521   
317 2020-03-30 -0.000267   -0.090794   -0.030213     0.094510   -0.056922   
318 2020-03-31  0.053204   -0.096715   -0.064271     0.092125   -0.073790   
319 2020-04-01  0.028533   -0.057384   -0.055833     0.119138   -0.096614   

     CAR_PBB.BE  CAR_MBK.DE  CAR_BBVA.MC  CAR_UNIE.XC  ...  CAR_ISP.MI  \
313    0.014712   -0.104005     0.014667    -0.092456  ...   -0.001591   
314    0.056486   -0.139074     0.038129    -0.089231  ...   -0.013510   
315    0.029179   -0.166463     0.025396    -0.103796  ...   -0.022336   
316    0.042614   -0.162275     0.015154    -0.050972  ...   -0.019933   
317   -0.0025

In [19]:
from scipy import stats
from scipy.stats import t

windows = {
    'CAR [-1,+1]': [-1, 1],
    'CAR [-1,+3]': [-1, 3],
    'CAR [0,+1]': [0, 1],
}


car_results = pd.DataFrame()
# Assume event_data contains the columns: 'bank', 'days_from_event', and 'Abnormal_Return'

for label, (start, end) in windows.items():

    window_data = find_data_in_relevent_days(event_data, start=start, end=end)

    print(f"{label}-window_data:", window_data)
    
#     assert window_data.equals(window_data2)
    
    # Calculate CAR for each bank
    car_per_bank = {}
    for bank in bank_list:
        if bank in window_data.columns:
            car_per_bank[bank] = window_data[f'Abnormal_{bank}'].sum()

    # Convert to DataFrame for merging
    car_per_bank_df = pd.DataFrame(list(car_per_bank.items()), columns=['Bank', label])
    
    # Merge the results into the car_results DataFrame
    if car_results.empty:
        car_results = car_per_bank_df
    else:
        car_results = pd.merge(car_results, car_per_bank_df, on='Bank', how='outer')
        
# Convert CAR results to percentages and format them to two decimal places
for column in car_results.columns[1:]:  # Skip the 'Bank' column
    car_results[column] = car_results[column] * 100  # Convert to percentage
    car_results[column] = car_results[column].apply(lambda x: f"{x:.2f}%")  # Format to two decimal places



CAR [-1,+1]-window_data:           Date       EBS    OBS.VI    RBI.VI   ARLN.HM    CBK.DE    PBB.BE  \
315 2020-03-26  0.491880  0.444000  0.496780  0.374594  0.471093  0.432316   
316 2020-03-27  0.338268  0.449819  0.405227  0.459381  0.385945  0.416488   
317 2020-03-30  0.507037  0.443000  0.399840  0.438714  0.374102  0.410073   

       MBK.DE   BBVA.MC   UNIE.XC  ...  CAR_SEB-A.ST  Expected_SHB-A.ST  \
315  0.432235  0.448903  0.446913  ...     -0.027623           0.461612   
316  0.409286  0.391638  0.454681  ...     -0.031055           0.403209   
317  0.467691  0.383570  0.469846  ...     -0.043757           0.457105   

     Abnormal_SHB-A.ST  CAR_SHB-A.ST  Expected_TFBANK.ST  Abnormal_TFBANK.ST  \
315          -0.010591     -0.007503            0.459140           -0.023712   
316           0.005537     -0.001966            0.403883           -0.004993   
317          -0.052928     -0.054894            0.454875           -0.017918   

     CAR_TFBANK.ST  Expected_NLB.IL  Abn

In [20]:
import numpy as np
from scipy.stats import t

# Create a copy of car_results to annotate the CAR values
car_results_copy = car_results.copy()

# Define significance levels and markers
alpha_levels = [0.01, 0.05, 0.10]
markers = ["***", "**", "*"]

# Iterate over each bank and window to annotate CAR values with significance markers
for bank in car_results['Bank'].unique():
    for label, (start, end) in windows.items():
        # Get abnormal returns for the specified window
        window_data = find_data_in_relevent_days(event_data, start=start, end=end)

        if f'Abnormal_{bank}' in window_data.columns:
            abnormal_returns = window_data[f'Abnormal_{bank}']

            # Calculate the standard deviation of abnormal returns
            std_dev = abnormal_returns.std(ddof=1)

            # Calculate the number of observations (days in the event window)
            n = len(abnormal_returns)

            if n > 1 and std_dev > 0:
                # Calculate the standard error (SE)
                standard_error = std_dev / np.sqrt(n)

                # Get the CAR value (removing percentage sign and converting to decimal)
                car_value = float(car_results.loc[car_results['Bank'] == bank, label].str.rstrip('%')) / 100

                # Calculate the t-statistic
                t_stat = car_value / standard_error

                # Annotate CAR value with significance markers based on different levels
                annotation = ""
                for alpha, marker in zip(alpha_levels, markers):
                    # Calculate the critical t-value for a two-tailed test
                    critical_value = t.ppf(1 - alpha / 2, df=n - 1)

                    # Determine if the t-statistic is greater than the critical value (significant)
                    if abs(t_stat) > critical_value:
                        annotation = marker
                        break  # Break at the highest significance level

                # Update the CAR value in the copy with the appropriate significance marker
                if annotation:
                    current_value = car_results_copy.loc[car_results_copy['Bank'] == bank, label].values[0]
                    car_results_copy.loc[car_results_copy['Bank'] == bank, label] = f"{current_value}{annotation}"
            else:
                # Detailed message for skipping due to insufficient data or zero variance
                if n <= 1:
                    print(f"Not enough observations for bank {bank} in window {label}. Skipping t-test.")
                elif std_dev == 0:
                    print(f"Zero variance for bank {bank} in window {label}. Skipping t-test.")
        else:
            print(f"No data available for bank {bank} in window {label}. Skipping t-test.")

# Display the annotated CAR results with significance markers
print("CAR Results with Significance Annotations:")
print(car_results_copy)


CAR Results with Significance Annotations:
         Bank CAR [-1,+1] CAR [-1,+3] CAR [0,+1]
0         EBS       1.45%       4.33%     -1.64%
1      OBS.VI       2.04%     5.38%**      3.44%
2      RBI.VI      -1.72%     -4.29%*     -5.36%
3     ARLN.HM      -4.40%      -1.94%      4.10%
4      CBK.DE     -8.99%*  -12.95%***     -9.87%
5      PBB.BE     -5.91%*    -4.48%**     -3.17%
6      MBK.DE      -1.09%      3.02%*      1.65%
7     BBVA.MC    -9.64%**  -13.09%***     -8.37%
8     UNIE.XC       5.12%      -0.52%      6.58%
9    AKTIA.HE       2.90%    6.44%***      2.27%
10  NDA-SE.ST      -1.00%      2.19%*     -1.22%
11   OMASP.HE      -0.90%       1.08%     -4.14%
12     BNP.PA      -4.61%  -10.07%***     -6.14%
13     CCN.PA       3.16%     6.28%**      4.58%
14     LLOY.L    -7.21%**  -20.00%***   -7.91%**
15     MTRO.L      -7.00%  -12.17%***    -10.79%
16      STB.L      13.79%    21.57%**     20.05%
17     STAN.L      -2.38%   -8.63%***     -2.42%
18    TATT.AT       5.79% 

In [21]:
# output_file_path = './Complete_CAR_Results_250_sp_EONIA.xlsx'
# car_results_copy.to_excel(output_file_path, index=False)

# output_file_path

In [22]:
# output_file_path = './Complete_CAR_Results_250_stoxx_EONIA.xlsx'
# car_results_copy.to_excel(output_file_path, index=False)

# output_file_path

In [23]:
# output_file_path = './Complete_CAR_Results_63_stoxx_EONIA.xlsx'
# car_results_copy.to_excel(output_file_path, index=False)

# output_file_path

In [24]:
# output_file_path = './Complete_CAR_Results_63_sp_EONIA.xlsx'
# car_results_copy.to_excel(output_file_path, index=False)

# output_file_path